# Module 8: 18 - Multi-Agents with CrewAI - Emulating Corporate Hierarchy
-------------------------------------------------------------------------
In this lesson, we will delve into the use of CrewAI to create and manage multi-agent systems that emulate a corporate hierarchy. CrewAI enables the specification of a custom manager agent or automatically creates one, requiring the specification of a manager language model (manager_llm). This manager agent is responsible for overseeing task execution, which includes planning, delegation, and validation.

In CrewAI, tasks are not pre-assigned to agents. Instead, the manager allocates tasks to agents based on their capabilities, reviews outputs, and assesses task completion. This hierarchical approach ensures efficient management and execution of tasks, reflecting a real-world corporate structure.

## Objectives
* Understand the structure and capabilities of CrewAI for creating hierarchical multi-agent systems.
* Implement and configure a manager agent to oversee task execution.
* Define agent capabilities and observe the task delegation process.
* Analyze the collaboration between agents under the supervision of a manager agent.

## What this session covers:
* Introduction to CrewAI and its role in emulating a corporate hierarchy in multi-agent systems.
* Setting up a manager agent and specifying a manager language model (manager_llm).
* Configuring agents with specific capabilities and enabling task delegation by the manager.
* Observing and analyzing the manager's role in planning, delegation, and validation of tasks.
* Demonstrating the hierarchical approach to task management and execution in a multi-agent system.

## Install Libraries

In [ ]:
#!pip install crewai

## Set OpenAI API Key & Model

In [6]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4-turbo'

## Define Agents

### SOC Analyst Agent

#### Define Tools

In [7]:
#from crewai_tools import tool
from langchain.tools import tool

@tool
def get_alert_tool(id: str) -> dict:
    """Returns alert information."""
    alerts = {
        "f96a7f37-e7f1-410f-85d3-93056e6601a5": {
            "alert": "Suspicious Login Attempt Detected",
            "severity": "High",
            "indicators": {
                "failed_login_attempts": 5,
                "successful_login_time": "2024-08-05T10:15:30Z",
                "unfamiliar_ip": "192.0.2.123",
                "geo_location": "Unknown Region",
                "affected_user": "jdoe@example.com"
            }
        },
        "a88c5e2a-2a4b-4848-b6e2-8f726a098b4e": {
            "alert": "Malware Detected in Cloud Storage",
            "severity": "Critical",
            "indicators": {
                "file_name": "invoice2024.exe",
                "file_hash": "d41d8cd98f00b204e9800998ecf8427e",
                "detection_time": "2024-08-05T12:45:00Z",
                "quarantine_status": "Quarantined",
                "malware_family": "RansomwareX",
                "affected_users": ["asmith@example.com", "bjones@example.com"]
            }
        },
        "d1b65937-0d8d-4b7e-8d2f-9e3412e0e1a4": {
            "alert": "Unusual Data Transfer Activity",
            "severity": "Medium",
            "indicators": {
                "data_volume": "150GB",
                "transfer_start_time": "2024-08-04T08:00:00Z",
                "transfer_end_time": "2024-08-04T09:30:00Z",
                "external_ip": "203.0.113.45",
                "source_database": "CustomerDB",
                "initiating_user": "jroe@example.com"
            }
        }
    }
    return alerts.get(id, "Alert ID not found")

In [8]:
@tool
def analyze_user_behavior_tool(user: str) -> dict:
    """Analyzes the behavior of a user account."""
    # Fake behavior analysis
    behavior_analysis = {
        "user": user,
        "login_history": [
            {"time": "2024-08-01T09:00:00Z", "ip": "198.51.100.23"},
            {"time": "2024-08-02T11:00:00Z", "ip": "198.51.100.23"},
            {"time": "2024-08-05T10:15:30Z", "ip": "192.0.2.123"}
        ],
        "anomalous_activities": ["Suspicious Login Attempt"]
    }
    return behavior_analysis

@tool
def analyze_data_transfer_tool(data_volume: str, external_ip: str) -> dict:
    """Analyzes unusual data transfer activities."""
    # Fake data transfer analysis result
    data_transfer_analysis = {
        "data_volume": data_volume,
        "external_ip": external_ip,
        "suspicious": True,
        "potential_exfiltration": True
    }
    return data_transfer_analysis

#### Define SOC Analyst Agent

In [9]:
from crewai import Agent

soc_analyst_agent = Agent(
  role='SOC Analyst',
  goal='Analyze security alerts.',
  backstory="""You work at organization X.
  Your expertise lies in investigating security alerts.""",
  verbose=True,
  allow_delegation=False,
  tools=[get_alert_tool, analyze_user_behavior_tool, analyze_data_transfer_tool]
)

### Threat Intelligence Analyst Agent

#### Define Tools

In [10]:
@tool
def analyze_ip_tool(ip: str) -> dict:
    """Enriches information and supports analysis of IP address for reputation and geolocation."""
    # Fake analysis result
    analysis_result = {
        "ip": ip,
        "reputation": "suspicious",
        "geo_location": "Unknown Region",
        "known_associations": ["malware distribution", "botnet activity"]
    }
    return analysis_result

#### Define TI Agent

In [11]:
ti_analyst_agent = Agent(
  role='Threat Intelligence Analyst',
  goal='Enrich indicators of compromise such as IP Addresses during alert investigations.',
  backstory="""You work at organization X.
  Your expertise lies in enriching indicators of compromise from security alerts.""",
  verbose=True,
  allow_delegation=False,
  tools=[analyze_ip_tool]
)

### Reverse Engineer Agent

#### Define Tools

In [12]:
@tool
def analyze_file_tool(file_hash: str) -> dict:
    """Analyzes a file for malware signatures and behavior."""
    # Fake file analysis result
    file_analysis_result = {
        "file_hash": file_hash,
        "detected_malware": "RansomwareX",
        "behavior": ["encrypts files", "contacts C2 server"]
    }
    return file_analysis_result

#### Define Reverse Engineer Agent

In [13]:
re_agent = Agent(
  role='Reverse Engineer',
  goal='Reverse files during alert investigations during alert investigations.',
  backstory="""You work at organization X.
  Your expertise lies in reverse engineering files from security alerts.""",
  verbose=True,
  allow_delegation=False,
  tools=[analyze_file_tool]
)

## Define Tasks

In [14]:
from crewai import Task

soc_analyst_task = Task(
  description="Investigate alert f96a7f37-e7f1-410f-85d3-93056e6601a5",
  expected_output="summary of alert investigation"
)

ti_analyst_task = Task(
  description="Help the investigation of security alerts if there is an IP address present as an indicator of compromise",
  expected_output="IP address enrichment"
)

re_task = Task(
  description="Help the investigation of security alerts if there is a file that needs to be reversed.",
  expected_output="reverse engineering report"
)

## Define Crew

In [15]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

crew = Crew(
    agents=[soc_analyst_agent,ti_analyst_agent,re_agent],
    tasks=[soc_analyst_task, ti_analyst_task, re_task],
    verbose=True,
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(
        temperature=0, model="gpt-4"
    ),
    telemetry = False,
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

 [2024-08-06 09:14:45][DEBUG]: == Working Agent: Crew Manager
 [2024-08-06 09:14:45][INFO]: == Starting Task: Investigate alert f96a7f37-e7f1-410f-85d3-93056e6601a5


> Entering new CrewAgentExecutor chain...
We need to start the investigation of the alert. The first step should be to understand the nature of the alert and gather as much information as possible. The SOC Analyst is the best person to start with as they are usually the first line of defense and have access to the alert details.

Action: Ask question to coworker
Action Input: {"question": "Can you provide the details of alert f96a7f37-e7f1-410f-85d3-93056e6601a5?", "context": "We need to investigate this alert and understand its nature. Please provide all the details you have including the time it was triggered, the system it was triggered on, the nature of the alert, and any other relevant information.", "coworker": "SOC Analyst"} 


> Entering new CrewAgentExecutor chain...
Thought: To provide a comprehensive report on 